In [1]:
import numpy as np
from scipy.optimize import curve_fit, root

# visualization tools
import matplotlib.pyplot as plt
from qiskit.visualization.bloch import Bloch

import qiskit.pulse as pulse
from qiskit.pulse import Schedule, Play, Acquire, DriveChannel, AcquireChannel, MemorySlot
from qiskit.pulse.pulse_lib import Gaussian, GaussianSquare, Waveform
from qiskit.compiler import assemble

from qiskit.ignis.characterization.calibrations import rabi_schedules, RabiFitter

# The pulse simulator
from qiskit.providers.aer import PulseSimulator

# function for constructing duffing models
from qiskit.providers.aer.pulse import duffing_system_model

<ipython-input-1-8e085403718e>:10: DeprecationWarning: the pulse_lib module is deprecated, pulse_lib is renamed to library
  from qiskit.pulse.pulse_lib import Gaussian, GaussianSquare, Waveform


Make a single 3d qubit

In [2]:
# cutoff dimension
dim_oscillators = 3

# frequencies for transmon drift terms, harmonic term and anharmonic term
# Number of oscillators in the model is determined from len(oscillator_freqs)
oscillator_freqs = [5.0e9]
anharm_freqs = [-0.33e9]

# drive strengths
drive_strengths = [0.02e9]

# specify coupling as a dictionary (qubits 0 and 1 are coupled with a coefficient 0.002e9)
coupling_dict = {}

# sample duration for pulse instructions 
dt = 1e-9

# create the model
one_qubit_model = duffing_system_model(dim_oscillators=dim_oscillators,
                                       oscillator_freqs=oscillator_freqs,
                                       anharm_freqs=anharm_freqs,
                                       drive_strengths=drive_strengths,
                                       coupling_dict=coupling_dict,
                                       dt=dt)

Do a $\pi$ pulse. The exact form the Hamiltonian (as can be seen in the `duffing_model_generators` documention) requires that the area under the pulse, multiplied by the drive strength, is `0.5`.

In [3]:
total_samples = 100
amp = 0.5 / (total_samples * dt * drive_strengths[0])

drive_pulse = Waveform(amp * np.ones(total_samples))
schedule = Schedule()
schedule |= Play(drive_pulse, DriveChannel(0))
schedule |= Acquire(total_samples, AcquireChannel(0), MemorySlot(0)) << schedule.duration

In [4]:
backend_sim = PulseSimulator()

qobj = assemble([schedule], 
                     backend=backend_sim, 
                     qubit_lo_freq=oscillator_freqs,
                     meas_level=1, 
                     meas_return='avg',
                     shots=1)

/Users/dpuzzuoli/Documents/projects/qiskit-terra/qiskit/compiler/assemble.py:320: RuntimeWarning: Dynamic rep rates not supported on this backend. rep_time will be used instead of rep_delay.
  warnings.warn(


In [5]:
results = backend_sim.run(qobj, one_qubit_model).result()

In [6]:
results.get_statevector(0)

array([ 1.27103260e-04+0.00706521j, -1.14928725e-02-0.99985547j,
       -3.94451879e-05-0.0103448j ])

In the above we see the system ends mostly in the excited state, but that the second excited state has non-trivial population.

If we run it again but restrict our model to only 2 levels, obviously no leakage will happen, but we also see a much higher fidelity pulse.

In [7]:
# cutoff dimension
dim_oscillators = 2

# frequencies for transmon drift terms, harmonic term and anharmonic term
# Number of oscillators in the model is determined from len(oscillator_freqs)
oscillator_freqs = [5.0e9]
anharm_freqs = [-0.33e9]

# drive strengths
drive_strengths = [0.02e9]

# specify coupling as a dictionary (qubits 0 and 1 are coupled with a coefficient 0.002e9)
coupling_dict = {}

# sample duration for pulse instructions 
dt = 1e-9

# create the model
one_qubit_model_2d = duffing_system_model(dim_oscillators=dim_oscillators,
                                       oscillator_freqs=oscillator_freqs,
                                       anharm_freqs=anharm_freqs,
                                       drive_strengths=drive_strengths,
                                       coupling_dict=coupling_dict,
                                       dt=dt)

In [8]:
results = backend_sim.run(qobj, one_qubit_model_2d).result()
results.get_statevector(0)

array([ 7.42173697e-07-2.49676911e-04j, -5.53883706e-10-9.99999969e-01j])

In [10]:
all(np.array([1,2]) == np.array([1,2]))

True

In [11]:
1 / np.array([1, 2])

array([1. , 0.5])

In [12]:
test = np.array([[1,2],[3,4]])

In [13]:
test**2

array([[ 1,  4],
       [ 9, 16]])